# Full Parameters Experiments

In [42]:
import pandas as pd
from collections import defaultdict
import numpy as np

FULL_CSV_PATH = "resultados/full_flags_experiment_results.csv"
SECOND_3_1_FULL_CSV_PATH = "resultados/second3_1_full_flags_experiment_results.csv"
CONC_CSV_PATH = "resultados/concurrent_experiment_results.csv"
SECOND_CONC_CSV_PATH = "resultados/second_concurrent_experiment_results.csv"

In [43]:
full_df = pd.read_csv(FULL_CSV_PATH)
second_full_df = pd.read_csv(SECOND_3_1_FULL_CSV_PATH)

full_df = pd.concat([full_df, second_full_df])

# Dictionary to store counts per (N, M)
full_grouped_counts = defaultdict(lambda: defaultdict(int))

# Optional: make sure N and M are present
if 'N' not in full_df.columns or 'M' not in full_df.columns:
    raise ValueError("Columns 'N' and 'M' must be present in the CSV")

for row in full_df.itertuples(index=False, name=None):
    row_dict = dict(zip(full_df.columns, row))
    n, m = row_dict['N'], row_dict['M']
    key = tuple(row_dict.items())  # full row as a hashable key
    full_grouped_counts[(n, m)][key] += 1

# Convert results to DataFrames
full_results = {}  # key = (N, M), value = DataFrame

for (n, m), counter in full_grouped_counts.items():
    # Convert dict of tuples back to DataFrame rows
    data = [dict(items) for items in counter.keys()]
    df = pd.DataFrame(data)
    df['frequency'] = list(counter.values())

    # Check if column A is null, NaN, or empty string (after stripping spaces)
    is_blank = df['GC_LOWER_BOUND'].isna() | (df['GC_LOWER_BOUND'].astype(str).str.strip() == '')

    # Create 'Finished' column: False if blank, else True
    df['Finished'] = ~is_blank

    # Now check convergence only where A is not blank
    df['Converged'] = np.where(
        df['Finished'],  # only evaluate where A is not blank
        df['GC_LOWER_BOUND'] <= df['GC_UPPER_BOUND'],  # True if A <= B
        False  # Otherwise (A is blank), set to False
    )
    full_results[(n, m)] = df


In [44]:
for key, value in full_results.items():
    print(f"N:{key[0]}, M:{key[1]}")

N:1, M:1
N:1, M:2
N:1, M:3
N:1, M:4
N:1, M:5
N:1, M:6
N:2, M:1
N:2, M:2
N:2, M:3
N:2, M:4
N:3, M:1


In [49]:
converged_df = full_results[(1,3)][full_results[(1,3)]['Converged'] == True]
converged_df.head()


,N,M,GC_LOWER_BOUND,GC_UPPER_BOUND,GC_LOWER_BOUND_REQUIRED_ITERATIONS,GC_UPPER_BOUND_REQUIRED_ITERATIONS,GC_SECONDS_TAKEN,METHOD,PRESOLVE,NUMERIC_FOCUS,FEASIBILITY_TOL,OPTIMALITY_TOL,frequency,Finished,Converged
0,1,3,0.10041,0.118533,30.0,32.0,0.851026,0,0.0,1.0,1.000000e-09,1.000000e-09,1,True,True
1,1,3,0.10041,0.118533,30.0,32.0,1.001720,0,0.0,1.0,1.000000e-07,1.000000e-09,1,True,True
2,1,3,0.10041,0.118533,30.0,37.0,0.757791,0,0.0,1.0,1.000000e-05,1.000000e-09,1,True,True
3,1,3,0.10041,0.118533,33.0,36.0,0.831646,0,0.0,1.0,1.000000e-02,1.000000e-09,1,True,True
4,1,3,0.10041,0.118533,50.0,42.0,1.135466,0,0.0,1.0,1.000000e-09,1.000000e-07,1,True,True


In [46]:
full_results[(1,5)].head()


,N,M,GC_LOWER_BOUND,GC_UPPER_BOUND,GC_LOWER_BOUND_REQUIRED_ITERATIONS,GC_UPPER_BOUND_REQUIRED_ITERATIONS,GC_SECONDS_TAKEN,METHOD,PRESOLVE,NUMERIC_FOCUS,FEASIBILITY_TOL,OPTIMALITY_TOL,frequency,Finished,Converged
0,1,5,205117.000000,-205117.0,26.0,17.0,2.141048,0,0.0,1.0,1.000000e-09,1.000000e-09,1,True,False
1,1,5,205117.000000,-205117.0,26.0,17.0,2.455851,0,0.0,1.0,1.000000e-07,1.000000e-09,1,True,False
2,1,5,205117.000000,-205117.0,27.0,15.0,2.852152,0,0.0,1.0,1.000000e-05,1.000000e-09,1,True,False
3,1,5,205116.999852,-205117.0,30.0,19.0,2.665814,0,0.0,1.0,1.000000e-02,1.000000e-09,1,True,False
4,1,5,205117.000000,-205117.0,26.0,17.0,2.083148,0,0.0,1.0,1.000000e-09,1.000000e-07,1,True,False


# Concurrent

In [47]:
conc_df = pd.read_csv(CONC_CSV_PATH)
second_conc_df = pd.read_csv(SECOND_CONC_CSV_PATH)

conc_df = pd.concat([conc_df, second_conc_df])
# Dictionary to store counts per (N, M)
conc_grouped_counts = defaultdict(lambda: defaultdict(int))

# Optional: make sure N and M are present
if 'N' not in conc_df.columns or 'M' not in conc_df.columns:
    raise ValueError("Columns 'N' and 'M' must be present in the CSV")

for row in conc_df.itertuples(index=False, name=None):
    row_dict = dict(zip(conc_df.columns, row))
    n, m = row_dict['N'], row_dict['M']
    key = tuple(row_dict.items())  # full row as a hashable key
    conc_grouped_counts[(n, m)][key] += 1

# Convert results to DataFrames
conc_results = {}  # key = (N, M), value = DataFrame

for (n, m), counter in conc_grouped_counts.items():
    # Convert dict of tuples back to DataFrame rows
    data = [dict(items) for items in counter.keys()]
    df = pd.DataFrame(data)
    df['frequency'] = list(counter.values())

    # Check if column A is null, NaN, or empty string (after stripping spaces)
    is_blank = df['GC_LOWER_BOUND'].isna() | (df['GC_LOWER_BOUND'].astype(str).str.strip() == '')

    # Create 'Finished' column: False if blank, else True
    df['Finished'] = ~is_blank

    # Now check convergence only where A is not blank
    df['Converged'] = np.where(
        df['Finished'],  # only evaluate where A is not blank
        df['GC_LOWER_BOUND'] <= df['GC_UPPER_BOUND'],  # True if A <= B
        False  # Otherwise (A is blank), set to False
    )
    conc_results[(n, m)] = df


In [48]:
for key, value in conc_results.items():
    print(f"N:{key[0]}, M:{key[1]}")

N:1, M:1
N:1, M:2
N:1, M:3
N:1, M:4
N:1, M:5
N:1, M:6
N:2, M:1
N:2, M:2
N:2, M:3
N:2, M:4
N:2, M:5
N:3, M:1
N:3, M:2
N:3, M:3
N:3, M:4
N:4, M:1
N:4, M:2
N:4, M:3
N:5, M:1
N:5, M:2
N:6, M:1
